In [52]:
this_file_name = 'sketching_configuration_mapping.ipynb'
import sys
import os
sys.path.insert(0, os.path.abspath(
    os.path.join(os.path.dirname(this_file_name), '../../src/') ))

from configuration_space_mapping import *
from utils import *

# Some simulation codes

First, retrieve all scene objects' configuration and position. 

In [53]:
client_id = connect_2_sim()
test_connection(client_id)

Connected to remote API server
Number of objects in the scene:  42


In [54]:
scene_objects = ['Cuboid_0','Cuboid_1','Cuboid_2',
                 'Cuboid_3','Cuboid_4','dr20']

In [55]:
listazinha = get_scene_objects_info(client_id, scene_objects)

In [56]:
for element in listazinha:
    a = round(element['object_position'][0],2)
    b = round(element['object_position'][1],2)
    c = round(element['object_position'][2],2)
    print([a,b,c])

[-1.72, 0.25, 0.25]
[-0.35, 1.5, 0.5]
[-1.52, -1.22, 0.5]
[-0.15, -0.1, 0.25]
[1.4, -0.15, 0.5]
[-2.02, 1.73, 0.15]


# Testing some random ideas and useful functions

In [57]:
info_list, robot_information = split_robot_from_info_list(listazinha,'dr20')

In [58]:
for element in info_list:
    print(element['object_name'])

Cuboid_0
Cuboid_1
Cuboid_2
Cuboid_3
Cuboid_4


In [59]:
handle_dr20 = get_object_handle(client_id,'dr20')
euler_orientation = (0,0,1.57)
sim.simxSetObjectOrientation(client_id,handle_dr20,-1,euler_orientation,sim.simx_opmode_blocking)

0

In [60]:
sim.simxGetObjectOrientation(client_id, handle_dr20,-1, sim.simx_opmode_blocking)

(0, [0.0019453406566753983, 2.178945578634739e-05, 1.570089340209961])

In [61]:
robot_pos, robot_ang = get_configuration(client_id, handle_dr20)
180*robot_ang[2]/3.14

90.00725351321469

In [62]:
get_normals_orientation(0)

(0, 1.570796326795, 3.14159265359, 4.7123889803850005)

In [63]:
for element in info_list:
    normals = get_normals_orientation(element['object_orientation'][2])
    # position_x = round
    print(f'{element["object_name"]}: {normals}')

Cuboid_0: (6.32791051020698e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)
Cuboid_1: (6.891567801451878e-14, 1.5707963267950689, 3.141592653590069, 4.712388980385069)
Cuboid_2: (2.999851555123585e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)
Cuboid_3: (2.4440758498663512e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)
Cuboid_4: (-1.0784487177348919e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)


In [64]:
x_min, y_min, z_min, x_max, y_max, z_max = get_bounding_box_corners_positions(client_id,listazinha[0]['object_handler'],"model")

In [65]:
# in order to test the map_local_coordinates_to_global_coordinates() power to map the corners local coordinates to global values,
# let's use the cuboid 0. It is global corners must be something like
# [-1.5, 0.5] [-2, 0.5] [-2, 0] [-1.5, 0]

cuboid_number = 0
local_c_cuboid = get_bounding_box_corners_local_coordinates(client_id, listazinha[cuboid_number]['object_handler'])
orientation_cuboid = listazinha[cuboid_number]['object_orientation']
orientation_cuboid.reverse() # notice that the orientation vector is reversed (theta is naturally the last, but we need to have it as the first value)
position_cuboid = listazinha[cuboid_number]['object_position']
map_local_coordinates_to_global_coordinates(local_c_cuboid, orientation_cuboid, position_cuboid)

[array([-1.47499979e+00,  5.00000359e-01,  6.06022016e-08]),
 array([-1.97499979e+00,  5.00000359e-01,  6.06466669e-08]),
 array([-1.97499979e+00,  3.58647658e-07,  5.86070880e-08]),
 array([-1.47499979e+00,  3.58647658e-07,  5.85626226e-08])]

we know there are 4 horizontal normal vectors. Two of them point to X and Y. The other two point to -X and -Y. 

How to know where X is pointing? Well, it is given by the Z Euler vector. 

Where does Y point to? to X+90º. And what about -X and -Y? We just need to add 180º to X and Y.

# Creating Configuration Map

1. Step
To orient the robot so its euler angles will be 0,0,0.
2. Step
For each obstacle, do the following:
- Get the bounding box vertices from $x_{min}, x_{max}, y_{min}, y_{max}, z_{min}, z_{max}$
- Map bounding box vertices (in object's reference frame to global reference frame)
- Test applicability using the normal's orientation (i think i'll need to check sine and cossine, or limit the orientation within [0,2pi] )
- Create Configuration Vertice 

In [66]:
cuboid_number = 0
orient_cuboid = info_list[cuboid_number]['object_orientation']
n_4_obj, n_1_obj, n_2_obj, n_3_obj = get_normals_orientation(orient_cuboid[2])

In [67]:
pi = 3.14159265359
orient_robot = robot_information['object_orientation']
n_4_robot, n_1_robot, n_2_robot, n_3_robot = get_normals_orientation(orient_robot[2])
neg_n_4_robot, neg_n_1_robot, neg_n_2_robot, neg_n_3_robot = n_4_robot+pi, n_1_robot+pi, n_2_robot+pi, n_3_robot+pi

In [68]:
normals_object = [n_1_obj, n_2_obj, n_3_obj, n_4_obj, n_1_obj]
normals_robot = [neg_n_4_robot, neg_n_1_robot, neg_n_2_robot, neg_n_3_robot, neg_n_4_robot]

In [74]:
normals_object = ['v1_obj','v2_obj','v3_obj','v4_obj','v1_obj']
normals_robot = ['v4_robo','v1_robo','v2_robo','v3_robo','v4_robo']
for it in range(0,len(normals_object)-1):
    print(f'A CONTACT: is {normals_robot[it]} between {normals_object[it]} and {normals_object[it+1]}?')
    print(f'B CONTACT: is {normals_object[it]} between {normals_robot[it]} and {normals_robot[it+1]}?')
    print(" ")

A CONTACT: is v4_robo between v1_obj and v2_obj?
B CONTACT: is v1_obj between v4_robo and v1_robo?
 
A CONTACT: is v1_robo between v2_obj and v3_obj?
B CONTACT: is v2_obj between v1_robo and v2_robo?
 
A CONTACT: is v2_robo between v3_obj and v4_obj?
B CONTACT: is v3_obj between v2_robo and v3_robo?
 
A CONTACT: is v3_robo between v4_obj and v1_obj?
B CONTACT: is v4_obj between v3_robo and v4_robo?
 


In [77]:
def is_angle_between(v_1,v,v_2):
    if v_2 < v_1:
        raise ValueError("angle order is wrong")

    cos_v1 = np.cos(v_1)
    sin_v1 = np.sin(v_1)

    cos_v = np.cos(v)
    sin_v = np.sin(v)

    cos_v2 = np.cos(v_2)
    sin_v2 = np.sin(v_2)

    if cos_v1 <= cos_v and cos_v <= cos_v2 and sin_v1 <= sin_v and sin_v <= sin_v2:
        return True
    else:
        return False

In [91]:
def convert_angle_to_0_2pi_interval(angle): # https://stackoverflow.com/questions/58627711/get-angle-into-range-0-2pi-python
    new_angle = np.arctan2(np.sin(angle), np.cos(angle))
    if new_angle < 0:
        new_angle = abs(new_angle) + 2 * (np.pi - abs(new_angle))
    return new_angle